In [1]:
# Imports and Setup
# First, we'll import the necessary libraries and load our environment variables.

from langchain_openai import ChatOpenAI
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain.memory import ChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from dotenv import load_dotenv
import os
load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')

In [2]:
# Chat History Management
def get_chat_history(store, session_id: str):
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

In [3]:
# Response Generation
def generate_response(chain_with_history, human_input: str, session_id: str, insights: str):
    response = chain_with_history.invoke(
        {"input": human_input, "insights": insights},
        config={"configurable": {"session_id": session_id}}
    )
    return response.content

In [4]:
# Reflection
def reflect(llm, store, session_id: str):
    reflection_prompt = ChatPromptTemplate.from_messages([
        ("system", "Based on the following conversation history, provide insights on how to improve responses:"),
        MessagesPlaceholder(variable_name="history"),
        ("human", "Generate insights for improvement:")
    ])
    reflection_chain = reflection_prompt | llm
    history = get_chat_history(store, session_id)
    reflection_response = reflection_chain.invoke({"history": history.messages})
    return reflection_response.content

In [5]:
# Learning
def learn(llm, store, session_id: str, insights: str):
    learning_prompt = ChatPromptTemplate.from_messages([
        ("system", "Based on these insights, update the agent's knowledge and behavior:"),
        ("human", "{insights}"),
        ("human", "Summarize the key points to remember:")
    ])
    learning_chain = learning_prompt | llm
    learned_points = learning_chain.invoke({"insights": insights}).content
    get_chat_history(store, session_id).add_ai_message(f"[SYSTEM] Agent learned: {learned_points}")
    return learned_points

In [6]:
# Self-Improving Agent Class
# Now we'll define our SelfImprovingAgent class that uses these functions.

class SelfImprovingAgent:
    def __init__(self):
        self.llm = ChatOpenAI(model="gpt-4o-mini", max_tokens=1000, temperature=0.7)
        self.store = {}
        self.insights = ""
        
        self.prompt = ChatPromptTemplate.from_messages([
            ("system", "You are a self-improving AI assistant. Learn from your interactions and improve your performance over time."),
            MessagesPlaceholder(variable_name="history"),
            ("human", "{input}"),
            ("system", "Recent insights for improvement: {insights}")
        ])
        
        self.chain = self.prompt | self.llm
        self.chain_with_history = RunnableWithMessageHistory(
            self.chain,
            lambda session_id: get_chat_history(self.store, session_id),
            input_messages_key="input",
            history_messages_key="history"
        )

    def respond(self, human_input: str, session_id: str):
        return generate_response(self.chain_with_history, human_input, session_id, self.insights)

    def reflect(self, session_id: str):
        self.insights = reflect(self.llm, self.store, session_id)
        return self.insights

    def learn(self, session_id: str):
        self.reflect(session_id)
        return learn(self.llm, self.store, session_id, self.insights)

In [7]:
# Example Usage
# Let's create an instance of our agent and interact with it to demonstrate its self-improving capabilities.

agent = SelfImprovingAgent()
session_id = "user_123"

# Interaction 1
print("AI:", agent.respond("What's the capital of France?", session_id))

# Interaction 2
print("AI:", agent.respond("Can you tell me more about its history?", session_id))

# Learn and improve
print("\nReflecting and learning...")
learned = agent.learn(session_id)
print("Learned:", learned)

# Interaction 3 (potentially improved based on learning)
print("\nAI:", agent.respond("What's a famous landmark in this city?", session_id))

# Interaction 4 (to demonstrate continued improvement)
print("AI:", agent.respond("What's another interesting fact about this city?", session_id))

AI: The capital of France is Paris. If you have any more questions or need information on a specific topic, feel free to ask!
AI: Certainly! Paris has a rich and complex history that spans over two millennia. Here are some key points:

1. **Ancient Origins**: Paris was originally a settlement known as Lutetia, founded by the Parisii, a Celtic tribe, around the 3rd century BC. It was situated on the Seine River.

2. **Roman Era**: By the 1st century AD, Lutetia became an important Roman city. Under Roman rule, it expanded significantly and developed infrastructure like baths, temples, and an amphitheater.

3. **Middle Ages**: After the fall of the Roman Empire, Paris continued to grow and became the capital of the Frankish kingdom under King Clovis I in the late 5th century. The city flourished during the medieval period, becoming a center of education and culture.

4. **The Renaissance**: The 16th century saw Paris emerge as a center of art and learning, influenced by the Renaissance. 